In [3]:
from main import  *

In [13]:
data = get_recently_played_tracks()

In [35]:

# def parse_streaming_history(data):
#     """ Get the recently played tracks info from the user.

#     Returns
#     -------
#     recently_played_tracks : list
#         A list of dictionaries with the recently played tracks info.
#     """
#     data = data
#     # create an empty list to store the parsed data
#     parsed_data = []
    
#     # loop through the items in the data list
#     for item in data["items"]:
#         # get the track object
#         track = item["track"]

#         # get the artist/artists data
#         artists = track["artists"]
#         artist_names = []
#         artist_ids = []
#         for artist in artists:
#             artist_names.append(artist["name"])
#             artist_ids.append(artist["id"])

#         # get the album data
#         album = track["album"]
#         album_name = album["name"]
#         album_id = album["id"]
#         album_type = album["album_type"]
#         release_date = album["release_date"]
#         total_tracks = album["total_tracks"]

#         # get the image with 300 height from the images list
#         images = album["images"]
#         image_300 = None
#         for image in images:
#             if image["height"] == 300:
#                 image_300 = image["url"]
#                 break

#         # get the track data
#         track_name = track["name"]
#         track_number = track["track_number"]
#         duration_ms = track["duration_ms"]
#         disc_number = track["disc_number"]
#         popularity = track["popularity"]
#         preview_url = track["preview_url"]

#         # get the track id from the uri by removing the spotify:track: prefix
#         track_id = track["uri"].replace("spotify:track:", "")

#         # get the context data
#         context = item["context"]
#         context_type = context["type"]
#         uri = context["uri"]

#         # create a dictionary to store the parsed data for each item
#         item_data = {
#             "artist_names": artist_names,
#             "artist_ids": artist_ids,
#             "album_name": album_name,
#             "album_id": album_id,
#             "album_type": album_type,
#             "release_date": release_date,
#             "total_tracks": total_tracks,
#             "image_url": image_300,
#             "track_name": track_name,
#             "track_number": track_number,
#             "duration_ms": duration_ms,
#             "disc_number": disc_number,
#             "popularity": popularity,
#             "preview_url": preview_url,
#             "track_id": track_id,
#             "context_type": context_type,
#             "uri": uri
#         }

#         # append the item data to the parsed data list
#         parsed_data.append(item_data)
#     return parsed_data

# parse_streaming_history(data)

In [5]:
import sqlite3

def store_user_streaming_history_to_database(streaming_history_json, database_name='streaming_history.db'):
    '''
    This function is divided into 2 parts:
    - Create the database and tables if they do not exist
    - Loop through the JSON data of streaming history and store the data to the database
    '''

    '''
    # Create the database and tables if they do not exist
    - connect to the database with the given name or create a new one if it does not exist
    - create a table for albums with columns for album_id, album_uri, album_name, album_type, release_date and total_tracks
    - create a table for artists with columns for artist_id, artist_uri and artist_name
    - create a table for tracks with columns for track_id, track_uri, track_name, track_number, duration_ms, disc_number, popularity, preview_url and image_url
    - create a table for streamings with columns for user_id, track_id, played_at, context_type and context_uri
    '''

    # connect to the database with the given name or create a new one if it does not exist
    # conn = sqlite3.connect(database_name)# connect to the database with the given name or create a new one if it does not exist
    with sqlite3.connect(database_name) as conn:
        cur = conn.cursor()

        # create a table for albums with columns for album_id, album_uri, album_name, album_type, release_date and total_tracks
        cur.execute('''CREATE TABLE IF NOT EXISTS albums(
                        album_id TEXT PRIMARY KEY,
                        album_uri TEXT NOT NULL,
                        album_name TEXT NOT NULL,
                        album_type TEXT NOT NULL,
                        release_date TEXT NOT NULL,
                        total_tracks INTEGER NOT NULL
                    )
                    ''')

        # create a table for artists with columns for artist_id, artist_uri and artist_name
        cur.execute('''CREATE TABLE IF NOT EXISTS artists(
                        artist_id TEXT PRIMARY KEY,
                        artist_uri TEXT NOT NULL,
                        artist_name TEXT NOT NULL
                    )
                    ''')

        # create a table for tracks with columns for track_id, track_uri, track_name, track_number, duration_ms, disc_number, popularity, preview_url and image_url
        # add foreign key constraints to reference the album_id from the albums table and the artist_id from the artists table
        cur.execute('''CREATE TABLE IF NOT EXISTS tracks (
                        track_id TEXT PRIMARY KEY,
                        track_uri TEXT NOT NULL,
                        track_name TEXT NOT NULL,
                        track_number INTEGER NOT NULL,
                        duration_ms INTEGER NOT NULL,
                        disc_number INTEGER NOT NULL,
                        popularity INTEGER NOT NULL,
                        preview_url TEXT,
                        image_url TEXT NOT NULL,
                        album_id TEXT NOT NULL,
                        artist_id TEXT NOT NULL,
                        FOREIGN KEY (album_id) REFERENCES albums(album_id),
                        FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
                    )
                    ''')

        # create a table for streamings with columns for user_id, track_id, played_at, context_type and context_uri
        # add foreign key constraints to reference the track_id from the tracks table
        cur.execute('''CREATE TABLE IF NOT EXISTS streamings(
                        user_id INTEGER NOT NULL,
                        track_id TEXT NOT NULL,
                        played_at TEXT NOT NULL,
                        context_type TEXT NOT NULL,
                        context_uri TEXT NOT NULL,
                        FOREIGN KEY (track_id) REFERENCES tracks(track_id)
                    )
                    '''
                    )

        '''
        # Loop through the JSON data of streaming history and store the data to the database
        - use a try-except block to handle any errors while inserting data to the tables
        - use a for loop to iterate over each item in the streaming_history_json list
        - extract the relevant data from each item such as album_id, artist_id, track_id etc.
        - check if the album_id already exists in the albums table using a SELECT query
        - if not, insert a new row to the albums table with the album data using an INSERT query
        - check if the artist_id already exists in the artists table using a SELECT query
        - if not, insert a new row to the artists table with the artist data using an INSERT query
        - check if the track_id already exists in the tracks table using a SELECT query
        - if not, insert a new row to the tracks table with the track data using an INSERT query
        - insert a new row to the streamings table with the user_id, track_id, played_at, context_type and context_uri using an INSERT query
        - commit the changes to the database after each iteration of the loop
        '''

        # use a try-except block to handle any errors while inserting data to the tables
        try:
            # use a for loop to iterate over each item in the streaming_history_json list
            for item in streaming_history_json:
                # extract the relevant data from each item such as album_id, artist_id, track_id etc.
                album_id = item['album_id']
                album_uri = item['album_uri']
                album_name = item['album_name']
                album_type = item['album_type']
                release_date = item['release_date']
                total_tracks = item['total_tracks']
                artist_ids = item['artist_ids']
                artist_names = item['artist_names']
                track_id = item['track_id']
                track_uri = item['track_uri']
                track_name = item['track_name']
                track_number = item['track_number']
                duration_ms = item['duration_ms']
                disc_number = item['disc_number']
                popularity = item['popularity']
                preview_url = item['preview_url']
                image_url = item['image_url']
                played_at = item['played_at']
                context_type = item['context_type']
                context_uri = item['context_uri']

                # check if the album_id already exists in the albums table using a SELECT query
                cur.execute('SELECT * FROM albums WHERE album_id=?', (album_id,))
                result = cur.fetchone()

                # if not, insert a new row to the albums table with the album data using an INSERT query
                if result is None:
                    cur.execute('INSERT INTO albums VALUES (:album_id, :album_uri, :album_name, :album_type, :release_date, :total_tracks)', 
                                {'album_id': album_id, 'album_uri': album_uri, 'album_name': album_name, 'album_type': album_type, 'release_date': release_date, 'total_tracks': total_tracks})

                # check if each artist in artist_ids already exists in artists table using SELECT queries
                # use zip function to iterate over both artist_ids and artist_names lists simultaneously
                for artist_id, artist_name in zip(artist_ids, artist_names):
                    cur.execute('SELECT * FROM artists WHERE artist_id=?', (artist_id,))
                    result = cur.fetchone()

                    # if not, insert new rows to artists table with each artist data using INSERT queries
                    if result is None:
                        cur.execute('INSERT INTO artists VALUES (?,?,?)', (artist_id, artist_uri, artist_name))

                    # check if each combination of track id and artist id already exists in tracks table using SELECT queries
                    cur.execute('SELECT * FROM tracks WHERE track_id=? AND artist_id=?', (track_id, artist_id))
                    result = cur.fetchone()

                    # if not, insert new rows to tracks table with each combination of track id and artist id data using INSERT queries
                    if result is None:
                        cur.execute('INSERT INTO tracks VALUES (?,?,?,?,?,?,?,?,?,?,?)',
                                    (track_id, track_uri, track_name, track_number, duration_ms, disc_number, popularity,
                                    preview_url, image_url, album_id, artist_id))

                    # insert new rows to streamings table with user_id, track_id, played_at, context_type and context_uri data using INSERT queries
                    cur.execute('INSERT INTO streamings VALUES (?,?,?,?,?)',
                                (user_id, track_id, played_at, context_type, context_uri))

                # commit changes to database after each iteration of loop
                conn.commit()

        except Exception as e:
            # print error message if any exception occurs
            print(e)

        finally:
            # close connection to database
            conn.close()


In [22]:
def parse_streaming_history():

  # Call the get_recently_played_tracks function and store the result in a variable.
  data = get_recently_played_tracks()

  # Create an empty list to store the parsed data.
  parsed_data = []

  # Loop through the items in the data list.
  for item in data["items"]:

    # Get the track object.
    track = item["track"]

    # Get the artist/artists data.
    artists = track["artists"]
    artist_names = []
    artist_ids = []
    for artist in artists:
      artist_names.append(artist["name"])
      artist_ids.append(artist["id"])

    # Get the album data.
    album = track["album"]
    album_name = album["name"]
    album_id = album["id"]
    album_type = album["album_type"]
    release_date = album["release_date"]
    total_tracks = album["total_tracks"]

    # Get the image with 300 height from the images list.
    images = album["images"]
    image_300 = None
    for image in images:
      if image["height"] == 300:
        image_300 = image["url"]
        break

    # Get the track data.
    track_name = track["name"]
    track_number = track["track_number"]
    duration_ms = track["duration_ms"]
    disc_number = track["disc_number"]
    popularity = track["popularity"]
    preview_url = track["preview_url"]

    # Get the played_at data.
    played_at = item["played_at"]

    # Get the track id from the uri by removing the spotify:track: prefix.
    track_id = track["uri"].replace("spotify:track:", "")

    # Get the context data.
    
    try:
      context = item["context"]
      context_type = context["type"]
      context_uri = context["uri"]
    except:
      context_uri = None
      context_type = None

    # Create a dictionary to store the parsed data for each item.
    item_data = {
      # Artist details
      "artist_names": artist_names,
      "artist_ids": artist_ids,
      # Album details
      "album_name": album_name,
      "album_id": album_id,
      "album_type": album_type,
      "release_date": release_date,
      "total_tracks": total_tracks,
      "image_url": image_300,
      # Track details
      "track_name": track_name,
      "track_number": track_number,
      "duration_ms": duration_ms,
      "disc_number": disc_number,
      "popularity": popularity,
      "preview_url": preview_url,
      "played_at": played_at,
      "track_id": track_id,
      # Context details
      "context_type": context_type,
      "context_uri": context_uri
    }

    # Append the item data to the parsed data list.
    parsed_data.append(item_data)

  # Return the parsed data list.
  return parsed_data

# Call the parse_streaming_history function and print the result.
# print(parse_streaming_history())


In [34]:

# def parse_streaming_history():
#     data = get_recently_played_tracks()
#     # create an empty list to store the parsed data
#     parsed_data = []
    
#     # loop through the items in the data list
#     for item in data["items"]:
#         # get the track object
#         track = item["track"]

#         # get the artist/artists data
#         artists = track["artists"]
#         artist_names = []
#         artist_ids = []
#         for artist in artists:
#             artist_names.append(artist["name"])
#             artist_ids.append(artist["id"])

#         # get the album data
#         album = track["album"]
#         album_name = album["name"]
#         album_id = album["id"]
#         album_type = album["album_type"]
#         release_date = album["release_date"]
#         total_tracks = album["total_tracks"]

#         # get the image with 300 height from the images list
#         images = album["images"]
#         image_300 = None
#         for image in images:
#             if image["height"] == 300:
#                 image_300 = image["url"]
#                 break

#         # get the track data
#         track_name = track["name"]
#         track_number = track["track_number"]
#         duration_ms = track["duration_ms"]
#         disc_number = track["disc_number"]
#         popularity = track["popularity"]
#         preview_url = track["preview_url"]

#         # get the track id from the uri by removing the spotify:track: prefix
#         track_id = track["uri"].replace("spotify:track:", "")

#         # get the context data
#         context = item["context"]
#         context_type = context["type"]
#         uri = context["uri"]

#         # create a dictionary to store the parsed data for each item
#         item_data = {
#             # artist details
#             "artist_names": artist_names,
#             "artist_ids": artist_ids,
#             # album details
#             "album_name": album_name,
#             "album_id": album_id,
#             "album_type": album_type,
#             "release_date": release_date,
#             "total_tracks": total_tracks,
#             "image_url": image_300,
#             #track details
#             "track_name": track_name,
#             "track_number": track_number,
#             "duration_ms": duration_ms,
#             "disc_number": disc_number,
#             "popularity": popularity,
#             "preview_url": preview_url,
#             "track_id": track_id,
#             # context details
#             "context_type": context_type,
#             "context_uri": uri
#             # timeplayed
#         }

#         # append the item data to the parsed data list
#         parsed_data.append(item_data)
#     return parsed_data

# data = parse_json(hist)

In [33]:
# from json_to_sqlite import *


# # define a function that takes a number as a parameter and prints the last n songs by each user and the songs in details with all the correct labels
# def print_last_played_songs(n):
#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()
#     # query the users table for all the user_ids and user_names
#     cur.execute("SELECT user_id, user_name FROM users")
#     users = cur.fetchall()
#     # create a table object with columns for streaming details
#     table = Table(show_header=True, header_style="bold magenta")
#     # sort the table by timestamp in ascending order
#     # table = table.sort_values(by="Time Since Played")
#     table.add_column("User")
#     table.add_column("Time Since Played")
#     table.add_column("Track URI")
#     table.add_column("Track Name")
#     table.add_column("Album Name")
#     table.add_column("Artist Name")
#     # loop through the users and add their streamings to the table rows
#     for user in users:
#         # extract the user_id and user_name from the tuple
#         user_id, user_name = user
#         # query the streamings table for the last n streamings of the user ordered by timestamp in descending order
#         cur.execute("SELECT track_id, timestamp FROM streamings WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?", (user_id, n))
#         streamings = cur.fetchall()
#         # loop through the streamings and add their details to the table rows
#         for streaming in streamings:
#             # extract the track_id and timestamp from the tuple
#             track_id, timestamp = streaming
#             # convert the timestamp to a datetime object
#             time_since_played = time_variation(int(timestamp))
#             # query the tracks table for the track data
#             cur.execute("SELECT track_uri, track_name, track_image_url, album_id, artist_id FROM tracks WHERE track_id = ?", (track_id,))
#             track_data = cur.fetchone()
#             # unpack the track data and store it in variables
#             track_uri, track_name, track_image_url, album_id, artist_id = track_data
#             # query the albums table for the album data
#             cur.execute("SELECT album_uri, album_name FROM albums WHERE album_id = ?", (album_id,))
#             album_data = cur.fetchone()
#             # unpack the album data and store it in variables
#             album_uri, album_name = album_data
#             # query the artists table for the artist data
#             cur.execute("SELECT artist_uri, artist_name FROM artists WHERE artist_id = ?", (artist_id,))
#             artist_data = cur.fetchone()
#             # unpack the artist data and store it in variables
#             artist_uri, artist_name = artist_data
#             # query the context table for the context data
#             cur.execute("SELECT context_uri, context_name, context_index FROM context WHERE context_uri IN (SELECT context_uri FROM tracks WHERE track_id = ?)", (track_id,))
#             context_data = cur.fetchone()
#             # unpack the context data and store it in variables
#             context_uri, context_name, context_index = context_data
#             # add a row to the table with the streaming details 
#             table.add_row(user_name, time_since_played, track_uri, track_name, album_name, artist_name)

#     # create a console object to print the table 
#     console = Console()
#     console.print(table)
#     # close the database connection
#     conn.close()

# print_last_played_songs(1)


In [ ]:
from rich.progress import track
def store_recently_played_tracks(spotify, username):
    # Get the user's playlists
    playlists = spotify.user_playlists(username)

    # Iterate over the playlists
    for playlist in playlists['items']:
        # Get the playlist's tracks
        tracks = spotify.user_playlist_tracks(username, playlist['id'])

        # Iterate over the tracks
        for track in tracks['items']:
            # Get the track's audio features
            audio_features = spotify.audio_features(track['track']['id'])

            # Iterate over the audio features
            for audio_feature in audio_features:
                # Store the audio features
                store_audio_features(audio_feature)

In [32]:
the_d = get_recently_played_tracks_info()

In [32]:
# def store_recently_played_tracks(database_name='MyStreamingHistory.db'):
#     """stores the recently played tracks in a SQLite database"""
#     # get the recently played tracks
#     recently_played_tracks = the_d
    
#     # create a connection to the SQLite database
#     conn = sqlite3.connect(database_name)
    
#     # create a cursor object
#     cur = conn.cursor()
    
#     # create a table if it does not exist
#     cur.execute("CREATE TABLE IF NOT EXISTS tracks (played_at TEXT, track_name TEXT)")
    
#     # get the last played track from the database
#     cur.execute("SELECT played_at FROM tracks ORDER BY played_at DESC LIMIT 1")
#     last_played_at = cur.fetchone()[0]
    
#     # reverse the new_recently_played_tracks
#     new_recently_played_tracks = recently_played_tracks[::-1]

#     # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
#     for i in range(len(new_recently_played_tracks)):
#         # get the difference between the last_played_at and the current track's played_at in hours
#         diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks[i]['played_at'])).total_seconds() / 3600
        
#         # if the difference is negative, then the current track is the one which was last played
#         if diff < 0:
#             # get the index of the new_recently_played_tracks
#             index = i
            
#             # start the new_recently_played_tracks from the index
#             new_recently_played_tracks = new_recently_played_tracks[index:]
            
#             # iterate through all the new_recently_played_tracks and print the track name and time
#             print("Newly added tracks:")
#             for track in new_recently_played_tracks:
#                 print(track['played_at'], track['track_name'])
                
#             # insert the new_recently_played_tracks to the database, use executemany() to avoid multiple queries
#             cur.executemany("INSERT INTO tracks VALUES (?, ?)", [(track['played_at'], track['track_name']) for track in new_recently_played_tracks])
            
#             # commit the changes to the database
#             conn.commit()
#             break
    
#     # close the connection to the database
#     conn.close()

# store_recently_played_tracks()

In [31]:
# 

In [30]:
# # connect to the database
# conn = sqlite3.connect('friends_activity.db')
# cur = conn.cursor()
# # create an empty dictionary to store the track_uri and their counts
# track_uri_counts = {}
# # query the tracks table for all the track_uri values
# cur.execute("SELECT track_uri FROM tracks")
# track_uris = cur.fetchall()
# # loop through the track_uris and update the dictionary with their counts
# for track_uri in track_uris:
#     # extract the track_uri value from the tuple
#     track_uri = track_uri[0]
#     # if the track_uri is already in the dictionary, increment its count by one
#     if track_uri in track_uri_counts:
#         track_uri_counts[track_uri] += 1
#     # otherwise, initialize its count to one
#     else:
#         track_uri_counts[track_uri] = 1
# # loop through the dictionary and print the track_uri and their counts if they are more than one
# for track_uri, count in track_uri_counts.items():
#     # if the count is more than one, it means there are multiple track uri in the table
#     if count >= 1:
#         print(track_uri, count)


In [29]:
# import time

# def get_current_time_in_millis():
#     # get the current time in seconds
#     current_time = time.time()
#     # convert it to milliseconds by multiplying by 1000
#     current_time_in_millis = int(current_time * 1000)
#     # return the result
#     return current_time_in_millis

# (get_current_time_in_millis() - 1684779485622)/1000/60

In [28]:
# def time_variation(timestamp):
#         # get the current time in seconds
#     current_time = time.time()
#     # convert it to milliseconds by multiplying by 1000
#     current_time_in_millis = int(current_time * 1000)
#     difference = current_time_in_millis- timestamp
#     minutes = difference/60000
#     # format the time difference as a string
#     if minutes < 1:
#         time_since_played = "Just now"
#     elif minutes == 1:
#         time_since_played = "1 minute ago"
#     elif minutes < 60:
#         time_since_played = f"{round(minutes)} minutes ago"
#     elif minutes == 60:
#         time_since_played = "1 hour ago"
#     elif minutes%60 == 0:
#         time_since_played = f"{minutes/60} hours ago"
#     elif minutes >  60:
#         time_since_played = f"{round(minutes/60)} hr {round(minutes%60)} min ago"
#     return time_since_played

# time_variation(1684779485622)

In [27]:
# import sqlite3
# # import the rich library
# from rich import print
# from rich.table import Table
# from rich.console import Console
# # import the datetime library
# from datetime import datetime


# # define a function that takes a number as a parameter and prints the last n songs by each user and the songs in details with all the correct labels
# def print_last_played_songs(n):
#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()
#     # query the users table for all the user_ids and user_names
#     cur.execute("SELECT user_id, user_name FROM users")
#     users = cur.fetchall()
#     # create a table object with columns for streaming details
#     table = Table(show_header=True, header_style="bold magenta")
#     # sort the table by timestamp in ascending order
#     # table = table.sort_values(by="Time Since Played")
#     table.add_column("User")
#     table.add_column("Time Since Played")
#     table.add_column("Track URI")
#     table.add_column("Track Name")
#     table.add_column("Album Name")
#     table.add_column("Artist Name")
#     # loop through the users and add their streamings to the table rows
#     for user in users:
#         # extract the user_id and user_name from the tuple
#         user_id, user_name = user
#         # query the streamings table for the last n streamings of the user ordered by timestamp in descending order
#         cur.execute("SELECT track_id, timestamp FROM streamings WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?", (user_id, n))
#         streamings = cur.fetchall()
#         # loop through the streamings and add their details to the table rows
#         for streaming in streamings:
#             # extract the track_id and timestamp from the tuple
#             track_id, timestamp = streaming
#             # convert the timestamp to a datetime object
#             time_since_played = time_variation(int(timestamp))
#             # query the tracks table for the track data
#             cur.execute("SELECT track_uri, track_name, track_image_url, album_id, artist_id FROM tracks WHERE track_id = ?", (track_id,))
#             track_data = cur.fetchone()
#             # unpack the track data and store it in variables
#             track_uri, track_name, track_image_url, album_id, artist_id = track_data
#             # query the albums table for the album data
#             cur.execute("SELECT album_uri, album_name FROM albums WHERE album_id = ?", (album_id,))
#             album_data = cur.fetchone()
#             # unpack the album data and store it in variables
#             album_uri, album_name = album_data
#             # query the artists table for the artist data
#             cur.execute("SELECT artist_uri, artist_name FROM artists WHERE artist_id = ?", (artist_id,))
#             artist_data = cur.fetchone()
#             # unpack the artist data and store it in variables
#             artist_uri, artist_name = artist_data
#             # query the context table for the context data
#             cur.execute("SELECT context_uri, context_name, context_index FROM context WHERE context_uri IN (SELECT context_uri FROM tracks WHERE track_id = ?)", (track_id,))
#             context_data = cur.fetchone()
#             # unpack the context data and store it in variables
#             context_uri, context_name, context_index = context_data
#             # add a row to the table with the streaming details 
#             table.add_row(user_name, time_since_played, track_uri, track_name, album_name, artist_name)
#     # create a console object to print the table 
#     console = Console()
#     console.print(table)

# print_last_played_songs(1)

In [26]:
# from rich import print
# from rich.table import Table
# from rich.console import Console

# def print_last_played_songs(num_results):
#  # connect to the database
#  conn = sqlite3.connect('friends_activity.db')
#  cur = conn.cursor()
#  # query the users table for all the user_ids and user_names
#  cur.execute("SELECT user_id, user_name FROM users")
#  users = cur.fetchall()
#  # loop through the users and print their names
#  for user in users:
#  # extract the user_id and user_name from the tuple
#  user_id, user_name = user
#  # print the user name in bold and green
#  print(f"[bold green]User: {user_name}[/bold green]")
#  # create a table object with columns for streaming details
#  table = Table(show_header=True, header_style="bold magenta")
#  table.add_column("User")
#  table.add_column("Timestamp")
#  table.add_column("Track URI")
#  table.add_column("Track Name")
#  # table.add_column("Track Image URL")
#  # table.add_column("Album URI")
#  table.add_column("Album Name")
#  # table.add_column("Artist URI")
#  table.add_column("Artist Name")
#  # table.add_column("Context URI")
#  # table.add_column("Context Name")
#  # table.add_column("Context Index")
#  # query the streamings table for the last 5 streamings of the user ordered by timestamp in descending order
#  cur.execute("SELECT user_id, track_id, timestamp FROM streamings WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?", (user_id, num_results))
#  streamings = cur.fetchall()
#  # loop through the streamings and add their details to the table rows
#  for streaming in streamings:
#  # extract the user_id, track_id, and timestamp from the tuple
#  user_id, track_id, timestamp = streaming
#  # query the tracks table for the track data
#  cur.execute("SELECT track_uri, track_name, track_image_url, album_id, artist_id FROM tracks WHERE track_id = ?", (track_id,))
#  track_data = cur.fetchone()
#  # unpack the track data and store it in variables
#  track_uri, track_name, track_image_url, album_id, artist_id = track_data
#  # query the albums table for the album data
#  cur.execute("SELECT album_uri, album_name FROM albums WHERE album_id = ?", (album_id,))
#  album_data = cur.fetchone()
#  # unpack the album data and store it in variables
#  album_uri, album_name = album_data
#  # query the artists table for the artist data
#  cur.execute("SELECT artist_uri, artist_name FROM artists WHERE artist_id = ?", (artist_id,))
#  artist_data = cur.fetchone()
#  # unpack the artist data and store it in variables
#  artist_uri, artist_name = artist_data
#  # query the context table for the context data
#  cur.execute("SELECT context_uri, context_name, context_index FROM context WHERE context_uri IN (SELECT context_uri FROM tracks WHERE track_id = ?)", (track_id,))
#  context_data = cur.fetchone()
#  # unpack the context data and store it in variables
#  context_uri, context_name, context_index = context_data
#  # add a row to the table with the streaming details 
#  table.add_row(user_name, timestamp, track_uri, track_name, album_name, artist_name
#  # , track_image_url, album_uri, artist_uri, context_uri, context_name, str(context_index)
#  )
#  # create a console object to print the table 
#  console = Console()
#  console.print(table)

# # close the database connection
# conn.close()

# print_last_played_songs(2)

In [25]:
# # import the rich library
# from rich import print
# from rich.table import Table
# from rich.console import Console

# # define a function that takes no arguments and prints the last played 5 songs by each user and the songs in details with all the correct labels
# def print_last_played_songs():
#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()
#     # query the users table for all the user_ids and user_names
#     cur.execute("SELECT user_id, user_name FROM users")
#     users = cur.fetchall()
#     # loop through the users and print their names
#     for user in users:
#         # extract the user_id and user_name from the tuple
#         user_id, user_name = user
#         # print the user name in bold and green
#         print(f"[bold green]User: {user_name}[/bold green]")
#         # create a table object with columns for streaming details
#         table = Table(show_header=True, header_style="bold magenta")
#         table.add_column("Timestamp")
#         table.add_column("Track URI")
#         table.add_column("Track Name")
#         # table.add_column("Track Image URL")
#         # table.add_column("Album URI")
#         table.add_column("Album Name")
#         # table.add_column("Artist URI")
#         table.add_column("Artist Name")
#         # table.add_column("Context URI")
#         # table.add_column("Context Name")
#         # table.add_column("Context Index")
#         # query the streamings table for the last 5 streamings of the user ordered by timestamp in descending order
#         cur.execute("SELECT track_id, timestamp FROM streamings WHERE user_id = ? ORDER BY timestamp DESC LIMIT 5", (user_id,))
#         streamings = cur.fetchall()
#         # loop through the streamings and add their details to the table rows
#         for streaming in streamings:
#             # extract the track_id and timestamp from the tuple
#             track_id, timestamp = streaming
#             # query the tracks table for the track data
#             cur.execute("SELECT track_uri, track_name, track_image_url, album_id, artist_id FROM tracks WHERE track_id = ?", (track_id,))
#             track_data = cur.fetchone()
#             # unpack the track data and store it in variables
#             track_uri, track_name, track_image_url, album_id, artist_id = track_data
#             # query the albums table for the album data
#             cur.execute("SELECT album_uri, album_name FROM albums WHERE album_id = ?", (album_id,))
#             album_data = cur.fetchone()
#             # unpack the album data and store it in variables
#             album_uri, album_name = album_data
#             # query the artists table for the artist data
#             cur.execute("SELECT artist_uri, artist_name FROM artists WHERE artist_id = ?", (artist_id,))
#             artist_data = cur.fetchone()
#             # unpack the artist data and store it in variables
#             artist_uri, artist_name = artist_data
#             # query the context table for the context data
#             cur.execute("SELECT context_uri, context_name, context_index FROM context WHERE context_uri IN (SELECT context_uri FROM tracks WHERE track_id = ?)", (track_id,))
#             context_data = cur.fetchone()
#             # unpack the context data and store it in variables
#             context_uri, context_name, context_index = context_data
#             # add a row to the table with the streaming details 
#             table.add_row(timestamp, track_uri, track_name, album_name, artist_name
#             # , track_image_url, album_uri, artist_uri, context_uri, context_name, str(context_index)
#             )
#         # create a console object to print the table 
#         console = Console()
#         console.print(table)


In [24]:
# # import json_to_sqlite.print_the_data_from_the_database
# import sqlite3
# def print_the_data_from_the_database():
#     '''
#     This function prints:
#     - all the data from the users table
#     - all the data from the albums table
#     - all the data from the artists table
#     - all the data from the tracks table
#     '''

#     #connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()

#     # define a function that takes a table name as an argument and prints all the data from that table
#     def print_table_data(table_name):
#         # select all the data from the table
#         cur.execute(f"SELECT * FROM {table_name}")
#         data = cur.fetchall()

#         # print the table name and the data
#         # print(f"{table_name.capitalize()}:")
#         for row in data:
#             print(row)

#     # call the function for each table
#     print("user_id, user_uri, user_name, user_image_url")
#     print_table_data("users")
#     print("album_id, album_uri, album_name")
#     print_table_data("albums")
#     print("artist_id, artist_uri, artist_name")
#     print_table_data("artists")
#     print("track_id, track_uri, track_name, track_image_url, album_id, artist_id")
#     print_table_data("tracks")
#     print("context_id, context_uri, context_name, context_index")
#     print_table_data("context")
#     print("streaming_id, user_id, track_id, timestamp")
#     print_table_data("streamings")

# print_the_data_from_the_database()

In [20]:
# # define a function that takes a user_id as an argument and returns all the details about that user from the database
# def get_user_details(user_id):
#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()
#     # create an empty dictionary to store the user details
#     user_details = {}
#     # query the users table for the user data
#     cur.execute("SELECT * FROM users WHERE user_id = ?", (user_id,))
#     user_data = cur.fetchone()
#     # if the query returns None, it means there is no user with that id in the table
#     if user_data is None:
#         return None
#     # otherwise, unpack the user data and store it in the dictionary
#     else:
#         user_id, user_url, user_name, user_image_url = user_data
#         user_details['user_id'] = user_id
#         user_details['user_url'] = user_url
#         user_details['user_name'] = user_name
#         user_details['user_image_url'] = user_image_url
#         # create an empty list to store the streamings of the user
#         user_details['streamings'] = []
#         # query the streamings table for the streamings of the user
#         cur.execute("SELECT * FROM streamings WHERE user_id = ?", (user_id,))
#         streamings_data = cur.fetchall()
#         # loop through the streamings data and get the track and context details for each streaming
#         for streaming_data in streamings_data:
#             # create an empty dictionary to store the streaming details
#             streaming_details = {}
#             # unpack the streaming data and store it in the dictionary
#             streaming_user_id, track_id, timestamp = streaming_data
#             streaming_details['timestamp'] = timestamp
#             # query the tracks table for the track data
#             cur.execute("SELECT * FROM tracks WHERE track_id = ?", (track_id,))
#             track_data = cur.fetchone()
#             # unpack the track data and store it in the dictionary
#             track_id, track_uri, track_name, track_image_url, album_id, artist_id = track_data
#             streaming_details['track_uri'] = track_uri
#             streaming_details['track_name'] = track_name
#             streaming_details['track_image_url'] = track_image_url
#             # query the albums table for the album data
#             cur.execute("SELECT * FROM albums WHERE album_id = ?", (album_id,))
#             album_data = cur.fetchone()
#             # unpack the album data and store it in the dictionary
#             album_id, album_uri, album_name = album_data
#             streaming_details['album_uri'] = album_uri
#             streaming_details['album_name'] = album_name
#             # query the artists table for the artist data
#             cur.execute("SELECT * FROM artists WHERE artist_id = ?", (artist_id,))
#             artist_data = cur.fetchone()
#             # unpack the artist data and store it in the dictionary
#             artist_id, artist_uri, artist_name = artist_data
#             streaming_details['artist_uri'] = artist_uri
#             streaming_details['artist_name'] = artist_name
#             # query the context table for the context data
#             cur.execute("SELECT * FROM context WHERE context_uri IN (SELECT context_uri FROM tracks WHERE track_id = ?)", (track_id,))
#             context_data = cur.fetchone()
#             # unpack the context data and store it in the dictionary
#             context_id, context_uri, context_name, context_index = context_data
#             streaming_details['context_uri'] = context_uri
#             streaming_details['context_name'] = context_name
#             streaming_details['context_index'] = context_index
#             # append the streaming details to the list of streamings of the user
#             user_details['streamings'].append(streaming_details)
#         # return the user details dictionary 
#         return user_details

# get_user_details(7)

{'user_id': 7,
 'user_url': 'spotify:user:gntab9tp1cc5qipthodlvvsm3',
 'user_name': 'suchita',
 'user_image_url': 'https://i.scdn.co/image/ab67757000003b829dc4867ae0babe8b2d882f5a',
 'streamings': [{'timestamp': '1684320742194',
   'track_uri': 'spotify:track:2Rlso2ZNV0PaWwUYeeBYxx',
   'track_name': 'Peppers (feat. Tommy Genesis)',
   'track_image_url': 'http://i.scdn.co/image/ab67616d0000b27359ae8cf65d498afdd5585634',
   'album_uri': 'spotify:album:5HOHne1wzItQlIYmLXLYfZ',
   'album_name': "Did you know that there's a tunnel under Ocean Blvd",
   'artist_uri': 'spotify:artist:00FQb4jTyendYWaN8pK0wa',
   'artist_name': 'Lana Del Rey',
   'context_uri': 'spotify:album:7JjgvQWsLpIAvwgj2QtxAP',
   'context_name': "The Bees Made Honey In The Lion's Skull",
   'context_index': 0},
  {'timestamp': '1684401969803',
   'track_uri': 'spotify:track:4aL0a88FhLISWmvanzprnA',
   'track_name': 'The City',
   'track_image_url': 'http://i.scdn.co/image/ab67616d0000b27304f21e12e67ba0dfd5db4fbf',
   'a

In [23]:
# # define a function that takes no arguments and returns all the user_id, user_url and name columns from the users table in the database
# def get_user_id_url_and_name():
#     # import the sqlite3 module
#     import sqlite3

#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()

#     # execute a SQL query to get the user_id, user_url and name columns from the users table
#     cur.execute("SELECT user_id, user_url, user_name FROM users")
    
#     # fetch all rows of data from the cursor object
#     user_data = cur.fetchall()

#     # close the connection
#     conn.close()

#     # return the user data as a list of tuples
#     return user_data

# get_user_id_and_name()



In [22]:
# # define a function that takes a user_id as an argument and returns all the details about that user from the database
# def get_user_details(user_id):
#     # import the sqlite3 module
#     import sqlite3

#     # connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()

#     # execute a SQL query to get all the data from the users table where the user_id matches the argument
#     cur.execute("SELECT * FROM users WHERE user_id = ?", (user_id,))
    
#     # fetch one row of data from the cursor object
#     user_data = cur.fetchone()

#     # close the connection
#     conn.close()

#     # return the user data as a tuple
#     return user_data

# print(get_user_details(7))

In [20]:
# def print_table_data(table_name):
#   # select all the data from the table
#   cur.execute(f"SELECT * FROM {table_name}")
#   data = cur.fetchall()

#   # print the table name and the data
#   # print(f"{table_name.capitalize()}:")
#   for row in data:
#     print(row)

# print_table_data("users")

In [19]:
# import csv

# # Open the rym music data CSV file for reading
# with open('rym_music_data.csv', 'r') as rym_file:
#     # Create a csv reader object
#     rym_reader = csv.reader(rym_file)
    
#     # Skip the header row
#     next(rym_reader)
    
#     # Open a new CSV file for writing
#     with open('allmusic_data.csv', 'w') as allmusic_file:
#         # Create a csv writer object
#         allmusic_writer = csv.writer(allmusic_file)
        
#         # Write the header row with the column names for allmusic.com
#         allmusic_writer.writerow(['Artist Name', 'Album Title', 'Release Year', 'Rating', 'Review'])
        
#         # Loop through each row in the rym music data CSV file
#         for row in rym_reader:
#             # Extract the relevant data from each row
#             rym_album_id = row[0]
#             artist_first_name = row[1]
#             artist_last_name = row[2]
#             artist_first_name_localized = row[3]
#             artist_last_name_localized = row[4]
#             album_title = row[5]
#             release_date = row[6]
#             rating = row[7]
#             ownership = row[8]
#             purchase_date = row[9]
#             media_type = row[10]
#             review = row[11]
            
#             # Format the data according to the allmusic.com data structure
#             # For example, you can concatenate the artist first and last names, or use the localized names if available
#             artist_name = artist_first_name + ' ' + artist_last_name
#             if artist_first_name_localized and artist_last_name_localized:
#                 artist_name = artist_first_name_localized + ' ' + artist_last_name_localized
            
#             # You can also extract the release year from the release date, or use a default value if not available
#             release_year = release_date.split('-')[0] if release_date else 'Unknown'
            
#             # You can also convert the rating from a numerical value to a star rating, or use a default value if not available
#             star_rating = int(rating) * '*' if rating else 'N/A'
            
#             # Write the formatted data to the new CSV file
#             allmusic_writer.writerow([artist_name, album_title, release_year, star_rating, review])


In [18]:
# import sqlite3
# import main

# # get the JSON data of friends' activity from the main module
# # friends_activity_json = main.get_friends_activity_json()

# main.store_user_data_to_database()



In [17]:
# print_the_data_from_the_database()

In [16]:

# # define a function to store the user data to a database
# def store_user_data_to_database(friends_activity_json, database_name='friends_activity.db'):
#     # connect to the database with the given name or create a new one if it does not exist
#     conn = sqlite3.connect(database_name)
#     cur = conn.cursor()

#     # create a table for users with columns for user_id, user_url, user_name and user_image_url
#     cur.execute('''CREATE TABLE IF NOT EXISTS users (
#         user_id INTEGER PRIMARY KEY,
#         user_url TEXT NOT NULL,
#         user_name TEXT NOT NULL,
#         user_image_url TEXT NOT NULL
#     )
#     ''')

#     # create a table for albums with columns for album_id, album_uri and album_name
#     cur.execute('''CREATE TABLE IF NOT EXISTS albums(
#         album_id INTEGER PRIMARY KEY,
#         album_uri TEXT NOT NULL,
#         album_name TEXT NOT NULL
#     )
#     ''')

#     # create a table for artists with columns for artist_id, artist_uri and artist_name
#     cur.execute('''CREATE TABLE IF NOT EXISTS artists(
#         artist_id INTEGER PRIMARY KEY,
#         artist_uri TEXT NOT NULL,
#         artist_name TEXT NOT NULL
#     )
#     ''')

#     # create a table for tracks with columns for track_id, track_uri, track_name, track_image_url, album_id and artist_id
#     # add foreign key constraints to reference the album_id and artist_id from the albums and artists tables respectively
#     cur.execute('''CREATE TABLE IF NOT EXISTS tracks (
#         track_id INTEGER PRIMARY KEY,
#         track_uri TEXT NOT NULL,
#         track_name TEXT NOT NULL,
#         track_image_url TEXT NOT NULL,
#         album_id INTEGER NOT NULL,
#         artist_id INTEGER NOT NULL,
#         FOREIGN KEY (album_id) REFERENCES albums(album_id),
#         FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
#     )
#     ''')

#     # create a table for context with columns for context_id, context_uri, context_name and context_index
#     cur.execute('''CREATE TABLE IF NOT EXISTS context(
#         context_id INTEGER PRIMARY KEY,
#         context_uri TEXT NOT NULL,
#         context_name TEXT NOT NULL,
#         context_index INTEGER NOT NULL
#     )
#     ''')

#     # create a table for streamings with columns for user_id, track_id and timestamp
#     # add foreign key constraints to reference the user_id and track_id from the users and tracks tables respectively
#     cur.execute('''CREATE TABLE IF NOT EXISTS streamings(
#         user_id INTEGER NOT NULL,
#         track_id INTEGER NOT NULL,
#         timestamp TEXT NOT NULL,
#         FOREIGN KEY (user_id) REFERENCES users(user_id),
#         FOREIGN KEY (track_id) REFERENCES tracks(track_id)
#     )
#     '''
#     )

#     # loop through the JSON data of friends' activity
#     for data in friends_activity_json['friends']:
#         # get the user data from the JSON object
#         print(data)
#         user_url = data['user']['uri']
#         user_name = data['user']['name']
#         user_image_url = data['user']['imageUrl']

#         # check if the user already exists in the users table by querying the user_url column
#         cur.execute("SELECT user_id FROM users WHERE user_url = ?", (user_url,))
#         user_id = cur.fetchone()

#         # if the query returns None, it means the user does not exist in the table
#         if user_id is None:
#             # insert a new row into the users table with the user data and get the generated user_id value
#             cur.execute("INSERT INTO users (user_url, user_name, user_image_url) VALUES (?, ?, ?)", (user_url, user_name, user_image_url))
#             conn.commit()
#             user_id = cur.lastrowid
#         else:
#             # if the query returns a tuple, it means the user already exists in the table and extract the first element of the tuple as the user_id value
#             user_id = user_id[0]

#         # get the album data from the JSON object
#         album_uri = data['track']['album']['uri']
#         album_name = data['track']['album']['name']

#         # check if the album already exists in the albums table by querying the album_uri column
#         cur.execute("SELECT album_id FROM albums WHERE album_uri = ?", (album_uri,))
#         album_id = cur.fetchone()

#         # if the query returns None, it means the album does not exist in the table
#         if album_id is None:
#             # insert a new row into the albums table with the album data and get the generated album_id value
#             cur.execute("INSERT INTO albums (album_uri, album_name) VALUES (?, ?)", (album_uri, album_name))
#             conn.commit()
#             album_id = cur.lastrowid
#         else:
#             # if the query returns a tuple, it means the album already exists in the table and extract the first element of the tuple as the album_id value
#             album_id = album_id[0]

#         # get the artist data from the JSON object
#         artist_uri = data['track']['artist']['uri']
#         artist_name = data['track']['artist']['name']

#         # check if the artist already exists in the artists table by querying the artist_uri column
#         cur.execute("SELECT artist_id FROM artists WHERE artist_uri = ?", (artist_uri,))
#         artist_id = cur.fetchone()

#         # if the query returns None, it means the artist does not exist in the table
#         if artist_id is None:
#             # insert a new row into the artists table with the artist data and get the generated artist_id value
#             cur.execute("INSERT INTO artists (artist_uri, artist_name) VALUES (?, ?)", (artist_uri, artist_name))
#             conn.commit()
#             artist_id = cur.lastrowid
#         else:
#             # if the query returns a tuple, it means the artist already exists in the table and extract the first element of the tuple as the artist_id value
#             artist_id = artist_id[0]

#         # get the track data from the JSON object
#         track_uri = data['track']['uri']
#         track_name = data['track']['name']
#         track_image_url = data['track']['imageUrl']

#         # check if the track already exists in the tracks table by querying the track_uri column
#         cur.execute("SELECT track_id FROM tracks WHERE track_uri = ?", (track_uri,))
#         track_id = cur.fetchone()

#         # if the query returns None, it means the track does not exist in the table
#         if track_id is None:
#             # insert a new row into the tracks table with the track data and get the generated track_id value
#             cur.execute("INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) VALUES (?, ?, ?, ?, ?)", (track_uri, track_name, track_image_url, album_id, artist_id))
#             conn.commit()
#             track_id = cur.lastrowid
#         else:
#             # if the query returns a tuple, it means the track already exists in the table and extract the first element of the tuple as the track_id value
#             track_id = track_id[0]

#         # get the context data from the JSON object
#         context_uri = data['track']['context']['uri']
#         context_name = data['track']['context']['name']
#         context_index = data['track']['context']['index']

#         # check if the context already exists in the context table by querying the context_uri column
#         cur.execute("SELECT context_id FROM context WHERE context_uri = ?", (context_uri,))
#         context_id = cur.fetchone()

#         # if the query returns None, it means the context does not exist in the table
#         if context_id is None:
#             # insert a new row into the context table with the context data and get the generated context_id value
#             cur.execute("INSERT INTO context (context_uri, context_name, context_index) VALUES (?, ?, ?)", (context_uri, context_name, context_index))
#             conn.commit()
#             context_id = cur.lastrowid
#         else:
#             # if the query returns a tuple, it means the context already exists in the table and extract the first element of the tuple as the context_id value
#             context_id = context_id[0]

#         # get the timestamp data from the JSON object
#         timestamp = data['timestamp']

#         # check if there is already a streaming with the same timestamp in the streamings table by querying the timestamp column
#         cur.execute("SELECT * FROM streamings WHERE timestamp = ?", (timestamp,))
#         streaming = cur.fetchone()

#         # if the query returns None, it means there is no streaming with the same timestamp in the table
#         if streaming is None:
#             # insert a new row into the streamings table with the user_id, track_id and timestamp values
#             cur.execute("INSERT INTO streamings (user_id, track_id, timestamp) VALUES (?, ?, ?)", (user_id, track_id, timestamp))
#             conn.commit()

In [15]:
# import sqlite3

# def print_the_data_from_the_database():
#     '''
#     This function prints:
#     - all the data from the users table
#     - all the data from the albums table
#     - all the data from the artists table
#     - all the data from the tracks table
#     '''

#     #connect to the database
#     conn = sqlite3.connect('friends_activity.db')
#     cur = conn.cursor()

#     # define a function that takes a table name as an argument and prints all the data from that table
#     def print_table_data(table_name):
#         # select all the data from the table
#         cur.execute(f"SELECT * FROM {table_name}")
#         data = cur.fetchall()

#         # print the table name and the data
#         print(f"{table_name.capitalize()}:")
#         for row in data:
#             print(row)

#     # call the function for each table
#     print_table_data("users")
#     print_table_data("albums")
#     print_table_data("artists")
#     print_table_data("tracks")
#     print_table_data("context")
#     print_table_data("streamings")

# print_the_data_from_the_database()

In [14]:

# friends_activity_json

In [14]:
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
# #
#     # Connect to the SQLite database.
#     cur = conn.cursor()
#     # create the users table with user_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_id INTEGER PRIMARY KEY,
#             user_uri TEXT NOT NULL UNIQUE,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     # create the tracks table with track_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_id INTEGER PRIMARY KEY,
#             track_uri TEXT NOT NULL UNIQUE,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     # create the contexts table with context_uri as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # create the listenings table with user_id and track_id as the foreign keys
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id INTEGER NOT NULL,
#             track_id INTEGER NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
#             FOREIGN KEY (user_id) REFERENCES users (user_id),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # print(friend[])
#         # add user to the database

#         # add user_uri, user_name and user_imageUrl if the user is not in the database

#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))

#         # get the user_id of the inserted user
#         cur.execute("""
#             SELECT user_id FROM users WHERE user_uri = ?
#         """, (friend['user']['uri'],))
#         user_id = cur.fetchone()[0]
#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # get the track_id of the inserted track
#         cur.execute("""
#             SELECT track_id FROM tracks WHERE track_uri = ?
#         """, (friend['track']['uri'],))
#         track_id = cur.fetchone()[0]
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         result = cur.fetchone()
#         if result:
#             # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
#             old_timestamp = result[0]
#             new_timestamp = friend['timestamp']
#             cur.execute("""
#                 UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
#             """, (new_timestamp, user_id, track_id, old_timestamp))
#         else:
#             # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
#             context_id = friend['track']['context']['uri']
#             timestamp = friend['timestamp']
#             cur.execute("""
#                 INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#                 VALUES (?, ?, ?, ?, 1)
#             """, (user_id, track_id, context_id, timestamp))
        
#         # previous_timestamps = cur.fetchall()
        
#     users = cur.fetchall()

#     for i in users:
#         print(i)


In [12]:
# #select all the data from the users table
# cur.execute("SELECT * FROM users")
# users = cur.fetchall()

# #print the user data
# print("Users:")
# for user in users:
#     print(user)

# #select all the data from the albums table
# cur.execute("SELECT * FROM albums")
# albums = cur.fetchall()

# #print the album data
# print("Albums:")
# for album in albums:
#     print(album)

# #select all the data from the artists table
# cur.execute("SELECT * FROM artists")
# artists = cur.fetchall()

# #print the artist data
# print("Artists:")
# for artist in artists:
#     print(artist)

# #select all the data from the tracks table
# cur.execute("SELECT * FROM tracks")
# tracks = cur.fetchall()

# #print the track data
# print("Tracks:")
# for track in tracks:
#     print(track)

# #select all the data from the context table
# cur.execute("SELECT * FROM context")
# context = cur.fetchall()

# #print the context data
# print("Context:")
# for c in context:
#     print(c)

# #select all the data from the streamings table
# cur.execute("SELECT * FROM streamings")
# streamings = cur.fetchall()

# #print the streaming data
# print("Streamings:")
# for streaming in streamings:
#     print(streaming)



In [13]:
# 

In [11]:
OLD CODE
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
# #
#     # Connect to the SQLite database.
#     cur = conn.cursor()
#     # create the users table with user_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_id INTEGER PRIMARY KEY,
#             user_uri TEXT NOT NULL UNIQUE,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     # create the tracks table with track_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_id INTEGER PRIMARY KEY,
#             track_uri TEXT NOT NULL UNIQUE,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     # create the contexts table with context_uri as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # create the listenings table with user_id and track_id as the foreign keys
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id INTEGER NOT NULL,
#             track_id INTEGER NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
#             FOREIGN KEY (user_id) REFERENCES users (user_id),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # print(friend[])
#         # add user to the database

#         # add user_uri, user_name and user_imageUrl if the user is not in the database
        

#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))

#         # get the user_id of the inserted user
#         cur.execute("""
#             SELECT user_id FROM users WHERE user_uri = ?
#         """, (friend['user']['uri'],))
#         user_id = cur.fetchone()[0]
#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # get the track_id of the inserted track
#         cur.execute("""
#             SELECT track_id FROM tracks WHERE track_uri = ?
#         """, (friend['track']['uri'],))
#         track_id = cur.fetchone()[0]
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         result = cur.fetchone()
#         if result:
#             # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
#             old_timestamp = result[0]
#             new_timestamp = friend['timestamp']
#             cur.execute("""
#                 UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
#             """, (new_timestamp, user_id, track_id, old_timestamp))
#         else:
#             # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
#             context_id = friend['track']['context']['uri']
#             timestamp = friend['timestamp']
#             cur.execute("""
#                 INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#                 VALUES (?, ?, ?, ?, 1)
#             """, (user_id, track_id, context_id, timestamp))
        
#         # previous_timestamps = cur.fetchall()
        
#     users = cur.fetchall()

#     for i in users:
#         print(i)


In [8]:
# # print the tables in the database
# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cur.fetchall())

# # add data to the tables
# for user in friends_activity_json['friends']:

# conn.commit()

# # print the tables in the database
# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cur.fetchall())
# ---------------------------------------------------------------------------
# KeyError                                  Traceback (most recent call last)
# c:\Users\saket\Documents\GitHub\Pyhton\Project Music\spotify api\test.ipynb Cell 3 in ()
#       5 # add data to the tables
#       6 for user in friends_activity_json['friends']:
# ----> 7     cur.execute('''INSERT INTO users(user_id, user_url, user_name, user_image_url) VALUES (?,?,?,?)''', (user['id'], user['url'], user['name'], user['imageUrl']))
#       8     for streaming in user['streamings']:
#       9         cur.execute('''INSERT INTO streamings(user_id, track_id, timestamp) VALUES (?,?,?)''', (user['id'], streaming['track']['id'], streaming['timestamp']))

# KeyError: 'id'


In [7]:

    # # # Connect to the SQLite database.
    # # cur = conn.cursor()
    # # # create the users table with user_id as the primary key
    # # cur.execute('''
    # #     CREATE TABLE IF NOT EXISTS users (
    # #         user_id INTEGER PRIMARY KEY,
    # #         user_uri TEXT NOT NULL UNIQUE,
    # #         user_name TEXT NOT NULL,
    # #         user_imageUrl TEXT NOT NULL
    # #     )
    # # ''')

    # # # create the tracks table with track_id as the primary key
    # # cur.execute('''
    # #     CREATE TABLE IF NOT EXISTS tracks (
    # #         track_id INTEGER PRIMARY KEY,
    # #         track_uri TEXT NOT NULL UNIQUE,
    # #         track_name TEXT NOT NULL,
    # #         track_imageUrl TEXT NOT NULL,
    # #         track_album_uri TEXT NOT NULL,
    # #         track_album_name TEXT NOT NULL,
    # #         track_artist_uri TEXT NOT NULL,
    # #         track_artist_name TEXT NOT NULL
    # #     )
    # # ''')

    # # # create the contexts table with context_uri as the primary key
    # # cur.execute('''
    # #     CREATE TABLE IF NOT EXISTS contexts (
    # #         context_uri TEXT PRIMARY KEY,
    # #         context_name TEXT NOT NULL
    # #     )
    # # ''')

    # # # create the listenings table with user_id and track_id as the foreign keys
    # # cur.execute('''
    # #     CREATE TABLE IF NOT EXISTS listenings (
    # #         user_id INTEGER NOT NULL,
    # #         track_id INTEGER NOT NULL,
    # #         context_id TEXT NOT NULL,
    # #         timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
    # #         track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
    # #         FOREIGN KEY (user_id) REFERENCES users (user_id),
    # #         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
    # #         FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
    # #     )
    # # ''')

    # # add friends activity to the database
    # for friend in friends_activity_json['friends']:
    #     # print(friend[])
    #     # add user to the database
    #     cur.execute("""
    #         INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
    #         VALUES (?, ?, ?) 
    #     """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
    #     # get the user_id of the inserted user
    #     cur.execute("""
    #         SELECT user_id FROM users WHERE user_uri = ?
    #     """, (friend['user']['uri'],))
    #     user_id = cur.fetchone()[0]
    #     # add track to the database
    #     cur.execute("""
    #         INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
    #         VALUES (?, ?, ?, ?, ?, ?, ?)
    #     """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
    #     # get the track_id of the inserted track
    #     cur.execute("""
    #         SELECT track_id FROM tracks WHERE track_uri = ?
    #     """, (friend['track']['uri'],))
    #     track_id = cur.fetchone()[0]
    #     # add context to the database
    #     cur.execute("""
    #         INSERT OR IGNORE INTO contexts (context_uri, context_name)
    #         VALUES (?, ?)
    #     """, (friend['track']['context']['uri'], friend['track']['context']['name']))
    #     # check if the user has listened to the same track before
    #     cur.execute("""
    #         SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
    #     """, (user_id, track_id))
    #     # check if the user has listened to the same track before
    #     cur.execute("""
    #         SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
    #     """, (user_id, track_id))
    #     result = cur.fetchone()
    #     if result:
    #         # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
    #         old_timestamp = result[0]
    #         new_timestamp = friend['timestamp']
    #         cur.execute("""
    #             UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
    #         """, (new_timestamp, user_id, track_id, old_timestamp))
    #     else:
    #         # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
    #         context_id = friend['track']['context']['uri']
    #         timestamp = friend['timestamp']
    #         cur.execute("""
    #             INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
    #             VALUES (?, ?, ?, ?, 1)
    #         """, (user_id, track_id, context_id, timestamp))
        
    #     # previous_timestamps = cur.fetchall()
        
    # users = cur.fetchall()

    # for i in users:
    #     print(i)


In [6]:

# def user_id_to_user_uri(users = ["gntab9tp1cc5qipthodlvvsm3"]):
#     """
#     Description
#     -----------
#     Convert a list of user ids to user uris.

#     Parameters
#     ----------
#     users : list
#         A list of user ids.

#     Returns
#     -------
#     user_uris : list
#         A list of user uris.
#     """
#     user_ids = ["spotify:user:" + user_id for user_id in user_ids]
#     return user_ids

In [4]:
# import main

In [3]:
# from main import *


# def create_file(filename):
#     # Create the files if they don't exist
#     if not os.path.exists(filename):
#         with open(filename, 'w') as f:
#             f.write('')


# def write_to_csv(filename, data):
#     # Write the data to the csv file
#     with open(filename, 'a', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(data)

# def main():
#     # Get the friends activity json
#     friends_activity_json = get_friends_activity_json()

#     # Get the current time
#     current_time = datetime.datetime.now()

#     # Iterate through the friends activity
#     for friend in friends_activity_json['friends']:
#         # Check if the user uri is the same as the user uri
#         if friend['user']['uri'] in user_id_to_user_uri():
#             # The id of the particular track the particular user is listening to
#             track_id = friend['track']['uri'].split(":")[-1]

#             # Check if the track is same as the previous track don't add it to the csv file
#             if track_id not in [i['track']['uri'].split(":")[-1] for i in friend_activity_csv['track_uri'].iloc[-10:]]:
#                 # Write the data to the csv file
#                 write_to_csv('friend_activity.csv', [friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

#                 # Get all the playlist tracks from the playlist
#                 the_playlist_tracks = get_playlist_tracks_retry(playlist_id)

#                 # If the track is not in the playlist add it to the playlist
#                 if track_id not in [i['track']['uri'].split(":")[-1] for i in the_playlist_tracks]:
#                     add_track_to_playlist(playlist_id, track_id)

#     # Wait for 30 seconds
#     time.sleep(30)


# if __name__ == '__main__':
#     main()


In [19]:


# def get_current_time():
#     # Get the current time.
#     return datetime.datetime.now()


# def check_if_track_is_new(track_id, friend_activity_csv):
#     # Check if the track is the same as the previous track.
#     if track_id not in [i['track_uri'].split(":")[-1] for i in friend_activity_csv['track_uri'].iloc[-10:]]:
#         return True
#     else:
#         return False


# def write_to_csv(filename, data):
#     # Write the data to the CSV file.
#     with open(filename, 'a', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(data)


# def get_playlist_tracks(playlist_id):
#     # Get all the playlist tracks.
#     while True:
#         try:
#             return sp.user_playlist_tracks(username, playlist_id)
#         except requests.exceptions.ConnectionError:
#             print('ConnectionError, retrying...')
#             time.sleep(30)


# def add_track_to_playlist(playlist_id, track_id):
#     # Add the track to the playlist.
#     while True:
#         try:
#             sp.user_playlist_add_tracks(username, playlist_id, [track_id])
#             break
#         except requests.exceptions.ConnectionError:
#             print('ConnectionError, retrying...')
#             time.sleep(30)


# def main():
#     # Get the friends activity data.
#     friends_activity_data = get_friends_activity_json()

#     # Get the current time.
#     current_time = get_current_time()

#     # Iterate through the friends activity data.
#     for friend in friends_activity_data['friends']:
#         # Check if the user URI is the same as the user URI.
#         if friend['user']['uri'] in user_ids:
#             # Get the track ID.
#             track_id = friend['track']['uri'].split(":")[-1]

#             # Check if the track is the same as the previous track.
#             if check_if_track_is_new(track_id, friend_activity_csv):
#                 # Write the data to the CSV file.
#                 write_to_csv('friend_activity.csv', [friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

#                 # Get all the playlist tracks.
#                 the_playlist_tracks = get_playlist_tracks(playlist_id)

#                 # If the track is not in the playlist, add it to the playlist.
#                 if track_id not in [i['track']['uri'].split(":")[-1] for i in the_playlist_tracks]:
#                     add_track_to_playlist(playlist_id, track_id)

#     # Wait for 30 seconds.
#     time.sleep(30)
# main()

In [18]:
# for i in friends_activity_json['friends']:
#     print(i['timestamp'])

In [2]:
# import sqlite3

# # connect to the database
# conn = sqlite3.connect('friends_activity.sqlite')

# # get a cursor to the database
# cur = conn.cursor()

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS users (
#         user_id INTEGER PRIMARY KEY,
#         user_uri TEXT NOT NULL,
#         user_name TEXT NOT NULL,
#         user_imageUrl TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS tracks (
#         track_id INTEGER PRIMARY KEY,
#         track_uri TEXT NOT NULL,
#         track_name TEXT NOT NULL,
#         track_imageUrl TEXT NOT NULL,
#         track_album_uri TEXT NOT NULL,
#         track_album_name TEXT NOT NULL,
#         track_artist_uri TEXT NOT NULL,
#         track_artist_name TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS contexts (
#         context_uri TEXT PRIMARY KEY,
#         context_name TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS listenings (
#         user_id INTEGER NOT NULL,
#         track_id INTEGER NOT NULL,
#         context_id TEXT NOT NULL,
#         timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#         track_listens_count INTEGER NOT NULL,
#         FOREIGN KEY (user_id) REFERENCES users (user_id),
#         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#         FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#     )
# ''')
# # add friends activity to the database
# for friend in friends_activity_json['friends']:
#     # check if the user already exists in the database
#     cur.execute("""
#         SELECT user_id FROM users WHERE user_uri = ?
#     """, (friend['user']['uri'],))
#     user_id = cur.fetchone()

#     # if the user does not exist, add them to the database
#     if user_id is None:
#         cur.execute("""
#             INSERT INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         user_id = cur.lastrowid

#     # check if the track already exists in the database
#     cur.execute("""
#         SELECT track_id FROM tracks WHERE track_uri = ?
#     """, (friend['track']['uri'],))
#     track_id = cur.fetchone()

#     # if the track does not exist, add it to the database
#     if track_id is None:
#         cur.execute("""
#             INSERT INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album_name'], friend['track']['artist']['uri'], friend['track']['artist_name']))
#         track_id = cur.lastrowid

#     # add listening to the database
#     cur.execute("""
#         INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#         VALUES (?, ?, NULL, ?, 1)
#     """, (user_id, track_id, friend['timestamp']))

# # commit the changes to the database
# conn.commit()

# # close the connection to the database
# conn.close()


In [7]:

# # Create a SQLite database file.
# conn = sqlite3.connect('friends_activity.sqlite')

# # Connect to the SQLite database.
# cur = conn.cursor()

# # print all the users from the database
# cur.execute("""
#     SELECT * FROM users
# """)
# users = cur.fetchall()

# print("users: \n user_uri, user_name, user_imageUrl")
# for i in users:
#     print(i)

# # print all the tracks from the database
# cur.execute("""
#     SELECT * FROM tracks
# """)
# tracks = cur.fetchall()

# print("tracks: \n track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name")
# for i in tracks:
#     print(i)

# # print all the contexts from the database
# cur.execute("""
#     SELECT * FROM contexts
# """)
# contexts = cur.fetchall()

# print("contexts: \n context_uri, context_name")
# for i in contexts:
#     print(i)

# # print all the listenings from the database
# cur.execute("""
#     SELECT * FROM listenings
# """)
# listenings = cur.fetchall()

# print("listenings: \n user_id, track_id, context_id, timestamp, track_listens_count")
# for i in listenings:
#     print(i)


# # Commit the changes to the database.
# conn.commit()

# # Close the database connection.
# conn.close()


In [8]:
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
#     # Connect to the SQLite database.
#     cur = conn.cursor()
#     # create the users table with user_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_id INTEGER PRIMARY KEY,
#             user_uri TEXT NOT NULL,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     # create the tracks table with track_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_id INTEGER PRIMARY KEY,
#             track_uri TEXT NOT NULL,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     # create the contexts table with context_uri as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # create the listenings table with user_id and track_id as the foreign keys
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id INTEGER NOT NULL,
#             track_id INTEGER NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL,
#             FOREIGN KEY (user_id) REFERENCES users (user_id),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # add user to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         # get the user_id of the inserted user
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             SELECT ?, ?, ? WHERE NOT EXISTS (
#                 SELECT 1 FROM users WHERE user_uri = ? AND user_name = ? AND user_imageUrl = ?
#             )
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl'], friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        
#         result = cur.fetchone()
#         if result:
#             user_id = result[0]

#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # get the track_id of the inserted track
#         cur.execute("""
#             SELECT track_id FROM tracks WHERE track_uri = ?
#         """, (friend['track']['uri'],))
#         track_id = cur.fetchone()[0]
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
        
#         # previous_timestamps = cur.fetchall()
        
#     users = cur.fetchall()

#     for i in users:
#         print(i)

# # No need to explicitly commit or close the connection as it is handled by the context manager.


In [9]:
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
#     # Connect to the SQLite database.
#     cur = conn.cursor()

#     # Check if the table exists.
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_uri TEXT PRIMARY KEY,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_uri TEXT PRIMARY KEY,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # Add a column for timestamps to the listenings table
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id TEXT NOT NULL,
#             track_id TEXT NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL,
#             FOREIGN KEY (user_id) REFERENCES users (user_uri),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_uri),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # add user to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (friend['user']['uri'], friend['track']['uri']))
        
#         previous_timestamps = cur.fetchall()
        
#         # if yes, update the timestamp column with the new value and increment the track_listens_count column by 1
#         if previous_timestamps:
#             cur.execute("""
#                 UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ?
#             """, (friend['timestamp'], friend['user']['uri'], friend['track']['uri']))
        
#         # if no, insert a new row with the user_id, track_id, context_id, timestamp and track_listens_count columns
#         else:
#             cur.execute("""
#                 INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#                 VALUES (?, ?, ?, ?, ?)
#             """, (friend['user']['uri'], friend['track']['uri'], friend['track']['context']['uri'], friend['timestamp'], 1))

#     # get all the users from the database
#     cur.execute("""
#         SELECT * FROM users
#     """)
#     users = cur.fetchall()

#     for i in users:
#         print(i)

# # No need to explicitly commit or close the connection as it is handled by the context manager.


In [1]:
# print(len("The blueprint is on a large, brownish yellow sheet of paper. The paper is slightly wrinkled and has a few folds in it. The drawing of the spaceship is in blue ink. The spaceship is a large, sleek, metallic craft. It is shaped like a teardrop, with a long, pointed nose and a wide, flat tail. The ship is covered in strange symbols and markings, which are written in an alien language that is unknown to humans. The text on the blueprint is also in blue ink."))

In [10]:
# friends_activity_json

In [11]:

# # Create a SQLite database file.
# conn = sqlite3.connect('friends_activity.sqlite')

# # Connect to the SQLite database.
# cur = conn.cursor()


# # select all data from listenings table
# # cur.execute("SELECT * FROM listenings")

# # adding my_dict to listenings table
# cur.execute("""
#     INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#     VALUES (?, ?, ?, ?, ?)
# """, (my_dict['user']['uri'], my_dict['track']['uri'], my_dict['context']['uri'], my_dict['timestamp'], 1))

# # commit the changes
# conn.commit()

# # close the connection
# conn.close()


# # If the result is None, create the table.

# # # create users table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS users (
# #         user_id INTEGER PRIMARY KEY,
# #         user_uri TEXT,
# #         user_name TEXT,
# #         user_imageUrl TEXT
# #     )
# # """)

# # # create tracks table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS tracks (
# #         track_id INTEGER PRIMARY KEY,
# #         track_uri TEXT,
# #         track_name TEXT,
# #         track_imageUrl TEXT,
# #         track_album_uri TEXT,
# #         track_album_name TEXT,
# #         track_artist_uri TEXT,
# #         track_artist_name TEXT
# #     )
# # """)

# # # create contexts table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS contexts (
# #         context_id INTEGER PRIMARY KEY,
# #         context_uri TEXT,
# #         context_name TEXT
# #     )
# # """)

# # # create listenings table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS listenings (
# #         listening_id INTEGER PRIMARY KEY,
# #         user_id INTEGER,
# #         track_id INTEGER,
# #         context_id INTEGER,
# #         timestamp INTEGER,
# #         track_listens_count INTEGER,
# #         FOREIGN KEY (user_id) REFERENCES users (user_id),
# #         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
# #         FOREIGN KEY (context_id) REFERENCES contexts (context_id)
# #     )
# # """)


In [12]:
# my_dict = {'timestamp': 1682175987406,
#    'user': {'uri': 'spotify:user:sonemic.com',
#     'name': 'sonemic.com',
#     'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
#    'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
#     'name': 'Revolve',
#     'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46',
#     'album': {'uri': 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw',
#      'name': 'Stoner Witch'},
#     'artist': {'uri': 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3',
#      'name': 'Melvins'},
#     'context': {'uri': 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S',
#      'name': 'Front Page Features | Sonemic Selects',
#      'index': 0}}}

In [13]:

# # create a cursor object
# cur = conn.cursor()

# # select all data from users table
# cur.execute("SELECT * FROM listenings")

# # fetch all rows as a list of tuples
# rows = cur.fetchall()
# print(rows)

# # loop through each row and print it
# for row in rows:
#     print(row)

# # # close the cursor and connection objects
# # cur.close()
# # conn.close()

In [30]:

# # Check if the table exists.
# cur.execute('''
#         SELECT name
#         FROM sqlite_master
#         WHERE type = 'table'
#         AND name = 'friends_activity'
#         ''')

# # Store the result of fetchone() in a variable.
# result = cur.fetchone()

# # If the result is None, create the table.
# if result is None:
#   print('Creating table...')
#   cur.execute('''
#         CREATE TABLE friends_activity (
#           timestamp INTEGER,
#           user_uri TEXT,
#           user_name TEXT,
#           user_imageUrl TEXT,
#           track_uri TEXT,
#           track_name TEXT,
#           track_imageUrl TEXT,
#           track_album_uri TEXT,
#           track_album_name TEXT,
#           track_artist_uri TEXT,
#           track_artist_name TEXT,
#           track_context_uri TEXT,
#           track_context_name TEXT,
#           track_context_index INTEGER,
#           track_listens_count INTEGER
#         )
#         ''')

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # fetch result from the database
# # result = cur.execute('''SELECT * FROM friends_activity''')
# result = cur.fetchall()


# # print the result
# for row in result:
#     print(row)


# # Insert the data into the table.
# for friend in friends_activity_json['friends']:
#   cur.execute(''' INSERT INTO friends_activity (timestamp,
#                                 user_uri,
#                                 user_name,
#                                 track_uri,
#                                 track_name,
#                                 track_imageUrl,
#                                 track_album_uri,
#                                 track_album_name,
#                                 track_artist_uri,
#                                 track_artist_name,
#                                 track_context_uri,
#                                 track_context_name,
#                                 track_listens)
#   VALUES (?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?, ?)''',
#        (friend['timestamp'],
#         friend['user']['uri'],
#         friend['user']['name'],
#         friend['user']['imageUrl'],
#         friend['track']['uri'],
#         friend['track']['name'],
#         friend['track']['imageUrl'],
#         friend['track']['album']['uri'],
#         friend['track']['album']['name'],
#         friend['track']['artist']['uri'],
#         friend['track']['artist']['name'],
#         friend['track']['context']['uri'],
#         friend['track']['context']['name'],
#         # friend['track']['context']['index'],
#         ))


# Close the connection to the SQLite database.
# conn.close()

In [14]:
# cur.execute('''SELECT * FROM friends_activity''')
# rows = cur.fetchall()
# # print(rows)
# # print(rows)
# for i in rows:
#     print(i)

In [19]:
# rows = cur.fetchall()
# print(rows)

[]


In [20]:
# print(rows)
# for i in rows:
#     print(i)

[]


In [15]:
# friends_activity_json

In [16]:
# data = (friend['timestamp'],
#         friend['user']['uri'],
#         friend['user']['name'],
#         friend['user']['imageUrl'],
#         friend['track']['uri'],
#         friend['track']['name'],
#         friend['track']['imageUrl'],
#         friend['track']['album']['uri'],
#         friend['track']['album']['name'],
#         friend['track']['artist']['uri'],
#         friend['track']['artist']['name'],
#         # friend['track']['context']['uri'],
#         # friend['track']['context']['name'],
#         # friend['track']['context']['index']
#         )
# for i in data:
#   print(i)

In [17]:
# friends_activity_json